In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [32]:
import nltk
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df=pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding='latin1')

In [5]:
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
col=['Label','ID','Date','Query','Username','Text']
df=df=pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',names=col, encoding='latin1')

In [7]:
df.head()

,Label,ID,Date,Query,Username,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df['Label'].unique()

array([0, 4])

In [9]:
drop_col = ['ID', 'Date', 'Query', 'Username']
df.drop(drop_col, axis=1, inplace=True)

In [10]:
df.head()

,Label,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [11]:
df['Label'].value_counts()

Label
0    800000
4    800000
Name: count, dtype: int64

### Preprocessing 

In [12]:
stop_words=stopwords.words('english')
stemmer=PorterStemmer()
re_cleaning= "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [16]:
def process_text(text):
    text = re.sub(re_cleaning, ' ', str(text).lower()).strip()
    tokens=[]
    for word  in text.split():
        if word  not in stop_words:
            stem_word=stemmer.stem(word)
            tokens.append(stem_word)
    
    
    return " ".join(tokens)

   


In [17]:
df['Text'] = df['Text'].apply(lambda x: process_text(x))

In [19]:
df.head()

,Label,Text
0,0,awww bummer shoulda got david carr third day
1,0,upset updat facebook text might cri result sch...
2,0,dive mani time ball manag save 50 rest go bound
3,0,whole bodi feel itchi like fire
4,0,behav mad see


### Data splitting

In [21]:
X=df['Text'].values
y=df['Label'].values

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [25]:
X_train

array(['ya quot like palm pre touchston charger readynow ye sound good beer readi prelaunch',
       'felt earthquak afternoon seem epicent', 'ruffl shirt like likey',
       ..., 'rather averag 32370',
       'pickin misstinayao waitin sadittysash 2 hurri odeee miss dem tabl talk 2nite lol bout fat',
       'home studi math wooot im go fail shit'], dtype=object)

### Modeling 

### Naive Bayes 

In [30]:
model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.76195
Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.78      0.77    159494
           4       0.77      0.74      0.76    160506

    accuracy                           0.76    320000
   macro avg       0.76      0.76      0.76    320000
weighted avg       0.76      0.76      0.76    320000



### Logistic Regression 


In [34]:
model = make_pipeline(CountVectorizer(), LogisticRegression(max_iter=1000))

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7730125
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.75      0.77    159494
           4       0.76      0.80      0.78    160506

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



In [37]:
# Error analysis
# Iterate over the instances where the predictions were incorrect
incorrect_indices = [i for i, (true_label, pred_label) in enumerate(zip(y_test, y_pred)) if true_label != pred_label]

print("Examples of incorrect predictions:")
for idx in incorrect_indices[:5]:  # Print the first 5 incorrect predictions as examples
    print(f"True label: {y_test[idx]}, Predicted label: {y_pred[idx]}, Text: {X_test[idx]}") 

Examples of incorrect predictions:
True label: 0, Predicted label: 4, Text: ahhh hope ok
True label: 0, Predicted label: 4, Text: cool tweet app razr 2
True label: 0, Predicted label: 4, Text: know famili drama lame hey next time u hang kim n u guy like sleepov whatev ill call u
True label: 0, Predicted label: 4, Text: go miss pastor sermon faith
True label: 4, Predicted label: 0, Text: zach make pee sit grown gay man
